In [1]:
!pip install geopandas
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 39.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [88]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt

import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#to make the interactive maps
import folium
from folium.plugins import FastMarkerCluster
import geopandas as gpd
from branca.colormap import LinearColormap

#to make the plotly graphs
import plotly.graph_objs as go
# import plotly.plotly as py
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

#text mining
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from wordcloud import WordCloud
import plotly.express as px

In [106]:
df = pd.read_csv('/content/drive/MyDrive/barcelona_listings.csv', index_col= "id", low_memory=False)

x = '''name                              19992 non-null object
host_id                           20030 non-null int64
host_name                         20026 non-null object
neighbourhood                     20030 non-null object
latitude                          20030 non-null float64
longitude                         20030 non-null float64
price                             20030 non-null int64
room_type                         20030 non-null object
property_type                     20030 non-null object
number_of_reviews                 20030 non-null int64
availability_365                  20030 non-null int64
accommodates                      20030 non-null int64
review_scores_rating              17391 non-null float64
cancellation_policy               19988 non-null object'''

x = x.split('\n')
columns = [i.split(' ')[0] for i in x]
df = df[columns].reset_index(drop=True)
df = df[df['cancellation_policy'].isin(['strict_14_with_grace_period', 'flexible', 'moderate'])]
df['cancellation_policy'] = df['cancellation_policy'].replace('strict_14_with_grace_period', 'strict')
df = df.dropna().reset_index(drop=True)
df['price'] = [float(i.replace("$","").replace(",","")) for i in df['price'].values]
df['room_type'] = df['room_type'].replace('Entire home/apt', 'All Rooms/Full Property')
k = [' with '.join(i) for i in df[['property_type','room_type']].values]
df['property_type vs room_type'] = [i if i in ['Apartment with Private room', 'Apartment with All Rooms/Full Property'] else 'Others' for i in k]
df['property_type vs room_type'].value_counts()
df.head(2)

,name,host_id,host_name,neighbourhood,latitude,longitude,price,room_type,property_type,number_of_reviews,availability_365,accommodates,review_scores_rating,cancellation_policy,property_type vs room_type
0,Flat with Sunny Terrace,71615,Mireia And Maria,Sant Martí,41.40889,2.18555,130.0,All Rooms/Full Property,Apartment,1,182,6,80.0,flexible,Apartment with All Rooms/Full Property
1,Huge flat for 8 people close to Sagrada Familia,71615,Mireia And Maria,La Sagrada Família,41.40420,2.17306,60.0,All Rooms/Full Property,Apartment,15,129,8,87.0,strict,Apartment with All Rooms/Full Property


In [90]:
df.columns

Index(['name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'price', 'room_type', 'property_type', 'number_of_reviews',
       'availability_365', 'accommodates', 'review_scores_rating',
       'cancellation_policy', 'property_type vs room_type'],
      dtype='object')

In [91]:
import plotly.graph_objects as go
import plotly.express as px

f = ['room_type', 'property_type', 'accommodates'][2]

k = df.copy()
k = k[k['price']<300]

fig = go.Figure()

fig.add_trace(go.Violin(
    x=k[f],
    y=k['price'],
    # points='all',
    name='Price',
    marker=dict(
        color='#ff3f3f',
        line=dict(color='#ff3f3f', width=2)
    ),
    box_visible=True,
    meanline_visible=True
))

fig.update_layout(
    title=f'Price by {f}',
    xaxis_title='Maximum Occupancy',
    yaxis_title='Price',
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white')
)

fig.show(renderer="colab")


In [58]:
df[['neighbourhood','latitude','longitude','price']].head(5)

,neighbourhood,latitude,longitude,price
0,Sant Martí,41.40889,2.18555,130.0
1,La Sagrada Família,41.40420,2.17306,60.0
2,Sant Martí,41.40560,2.19821,33.0
3,Sant Martí,41.41203,2.22114,210.0
4,Vila de Gràcia,41.40145,2.15645,45.0


In [62]:
avg_prices.head()

,neighbourhood,latitude,longitude,price
0,Camp d'en Grassot i Gràcia Nova,41.405750,2.166352,81.229508
1,Ciutat Vella,41.382105,2.175675,84.852700
2,Dreta de l'Eixample,41.393359,2.168927,253.161429
3,Eixample,41.391511,2.165500,153.611491
4,El Besòs i el Maresme,41.415414,2.215388,48.235294


In [83]:
import geopandas as gpd
import json

# Load the GeoJSON file into a GeoDataFrame
gdf = gpd.read_file('https://raw.githubusercontent.com/martgnz/bcn-geodata/master/area-estadistica-basica/area-estadistica-basica.geojson')

# Print the first few rows of the GeoDataFrame
x = ['ID_ANNEX', 'ANNEXDESCR', 'ID_THEME', 'THEME_DESCR', 'ID_SET',
'SET_DESCR', 'ID_SUBSET', 'SUBSET_DESCR', 'ID_ELEMENT', 'ELEMENT_DESCR',
'LEVEL', 'NDESCR_CA', 'NDESCR_ES', 'NDESCR_EN', 'TERM', 'DISTRICT',
'NEIGHBOURHOOD', 'AEB', 'SEC_CENSUS', 'GRANBARRI', 'ZUA', 'AREA_I', 'LITERAL',
'PERIMETER', 'AREA', 'ORD_REPRESENT', 'CODI_UA', 'TIPUS_UA', 'NOM', 'WEB1',
'WEB2', 'WEB3', 'DOCUMENTA', 'RANGE_SCALE', 'TIPUS_POLYGON', 'GRUIX_ID',
'GRUIXDIMEN', 'ESTIL_ID', 'ESTIL_QGIS', 'VALOR1QGIS', 'VALOR2QGIS',
'COL_FILLED', 'FCOL_DESCR', 'FHEX_COLOR', 'COL_DESCR', 'HEX_COLOR7',
'geometry']

gdf.columns = [i.lower() for i in x]

# geojson_str = gdf.to_json()

with open('data.geojson' , 'w') as file:
    file.write(gdf.to_json())

In [85]:
gdf = gpd.read_file('/content/data.geojson')
gdf.head(2)

,id,id_annex,annexdescr,id_theme,theme_descr,id_set,set_descr,id_subset,subset_descr,id_element,...,estil_id,estil_qgis,valor1qgis,valor2qgis,col_filled,fcol_descr,fhex_color,col_descr,hex_color7,geometry
0,0,01,Grup - I,0104,Unitats Administratives,010414,Àrea estadística bàsica,01041401,Àrea estadística bàsica,0104140101,...,0,Sòlid,0,0,18,NULL,#FFA800,?????,#CC6804,"POLYGON ((2.17773 41.37515, 2.17777 41.37513, ..."
1,1,01,Grup - I,0104,Unitats Administratives,010414,Àrea estadística bàsica,01041401,Àrea estadística bàsica,0104140101,...,0,Sòlid,0,0,18,NULL,#FFA800,?????,#CC6804,"POLYGON ((2.17220 41.37692, 2.17265 41.37681, ..."


In [ ]:
import folium

# Create map object centered on Barcelona
m = folium.Map(location=[41.3851, 2.1734], zoom_start=12)

# Add markers for each hotel location
for _, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                  popup=f"Price: {row['price']} €",
                  icon=folium.Icon(color='blue', icon='hotel')).add_to(m)

# Show map
m


In [117]:
import plotly.express as px

# create the scatter map
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="price",
                        color_continuous_scale=["black", "pink"], zoom=11)

# update the layout to a grey background with white mapbox
fig.update_layout(
    mapbox_style="carto-positron",
    mapbox=dict(
        accesstoken='your-token-here',
        bgcolor="#f7f7f7",
        bearing=0,
        center=dict(
            lat=41.3851,
            lon=2.1734
        ),
        pitch=0,
        zoom=11
    )
)

fig.show(renderer="colab")

ValueError: ignored

In [125]:
import pandas as pd
import plotly.express as px

# Define the map center and zoom level
map_center = {"lat": 41.3851, "lon": 2.1734}
map_zoom = 11

df = df[['neighbourhood','latitude','longitude','price']]
df = df[df['price']<300]

import pandas as pd
import plotly.express as px

# create the scatter map
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", color="price",
                        color_continuous_scale=["black", "red"], zoom=11)

# update the layout to a black background with white mapbox
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox=dict(
        accesstoken='your-token-here',
        bearing=0,
        center=dict(
            lat=41.3851,
            lon=2.1734
        ),
        pitch=0,
        zoom=11
    )
)

fig.show(renderer="colab")

In [92]:
import pandas as pd
import plotly.express as px

# Load data
data = df[['neighbourhood','latitude','longitude','price']]

# Compute average prices by neighborhood
avg_prices = data.groupby('neighbourhood').mean().reset_index()

# Create the choropleth map
fig = px.choropleth_mapbox(avg_prices, 
                           geojson='/content/data.geojson', 
                           locations='neighbourhood', 
                           color='price',
                           color_continuous_scale='reds',
                           mapbox_style='carto-positron',
                           zoom=11,
                           center={'lat': 41.390205, 'lon': 2.154007},
                           opacity=0.7,
                           hover_name='neighbourhood',
                           hover_data=['price']
                          )

fig.update_layout(
    title='Average Hotel Prices by Neighborhood in Barcelona',
    mapbox_accesstoken='your_mapbox_token_here',
    margin={'l':0,'r':0,'t':30,'b':0},
    paper_bgcolor='white',
    font=dict(color='black'),
)

fig.show(renderer="colab")

In [41]:
# k = listings[listings['neighbourhood']=='Eixample']
k = df.copy()
k = k[k['price']<300]
fig = px.violin(k, x="cancellation_policy", y="price", points="all", box=True)
fig.show(renderer="colab")

In [5]:
k[['accommodates','price']].corr()

,accommodates,price
accommodates,1.000000,0.658558
price,0.658558,1.000000


In [38]:
df.columns

Index(['name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'price', 'room_type', 'property_type', 'number_of_reviews',
       'availability_365', 'accommodates', 'review_scores_rating',
       'cancellation_policy', 'property_type vs room_type'],
      dtype='object')

In [39]:
listings = df.copy()
k = listings['neighbourhood'].value_counts().sort_values(ascending=False).index[:10]
k = listings[listings['neighbourhood'].isin(k)]
k = k[k['price']<300]
fig = px.box(k, x="room_type", y="price", points="all")
fig.show(renderer="colab")

# 2.1 Neighbourhoods
Neighbourhood "La Verneda i La Pau" holds most listings, and altogether eight neigbourhoods have over 10.

In [7]:
# feq = listings['neighbourhood'].value_counts().sort_values(ascending=True).iloc[:20]
# feq.plot.barh(figsize=(7, 5), color='b', width=1)
# plt.title("Number of listings by neighbourhood", fontsize=10)
# plt.xlabel('Number of listings', fontsize=10)
# plt.show()

import plotly.express as px

listings = df.copy()
feq = listings['neighbourhood'].value_counts().sort_values(ascending=True).iloc[-10:]

fig = px.bar(feq, x=feq.values, y=feq.index, orientation='h', color=feq.values,
             color_continuous_scale=px.colors.sequential.Blues)

fig.update_layout(title="Number of listings by neighbourhood", xaxis_title="Number of listings",
                  yaxis_title="Neighbourhood", font=dict(size=10))

fig.show(renderer="colab")

Below, you can see that most listings are in the city centre. This map is interactive, and you can zoom-in on the clusters to eventually find the individual locations of the listings.

Note: In a later version I made a map on the average daily price per neighoudhood (section 4.1). I think this map would also be better at this point (as it uses the neighbourhood shapefile), but as my main goal of this kernel is to learn as much as possible I left the map below unchanged as FastMarkerCluster seems useful for the future.



In [8]:
lats2018 = listings['latitude'].tolist()
lons2018 = listings['longitude'].tolist()
locations = list(zip(lats2018, lons2018))

map1 = folium.Map(location=[41.389513, 2.165977], zoom_start=14)
FastMarkerCluster(data=locations).add_to(map1)
map1

## Data processing

In [9]:
df.columns

Index(['name', 'host_id', 'host_name', 'neighbourhood', 'latitude',
       'longitude', 'price', 'room_type', 'property_type', 'number_of_reviews',
       'availability_365', 'accommodates', 'review_scores_rating',
       'cancellation_policy', 'property_type vs room_type'],
      dtype='object')

In [10]:
df.to_csv('barcelona_data.csv', index=False)

In [11]:
df = pd.read_csv('barcelona_data.csv')

imp_fea = ['neighbourhood', 'latitude',
       'longitude', 'price', 'room_type', 'property_type', 'number_of_reviews',
       'availability_365', 'accommodates', 'review_scores_rating',
       'cancellation_policy', 'property_type vs room_type']
df[imp_fea]
cat_fea = ['neighbourhood','room_type', 'property_type', 'cancellation_policy', 'property_type vs room_type']
num_fea = list(set(imp_fea)-set(cat_fea))

### Fixing neighbourhood

In [12]:
top_hood = df['neighbourhood'].value_counts()[:30].index
df['neighbourhood'] = [i if i in top_hood else 'other' for i in df['neighbourhood']]
df['neighbourhood'].value_counts()

Eixample                           2924
Ciutat Vella                       1704
Sants-Montjuïc                     1388
other                               910
Sant Martí                          888
Dreta de l'Eixample                 700
Gràcia                              667
El Raval                            627
La Nova Esquerra de l'Eixample      386
Sant Antoni                         383
El Gòtic                            381
Vila de Gràcia                      366
El Poble-sec                        358
La Sagrada Família                  343
L'Antiga Esquerra de l'Eixample     315
Sant Pere/Santa Caterina            282
Sarrià-Sant Gervasi                 264
Les Corts                           250
Horta-Guinardó                      220
el Fort Pienc                       183
El Poblenou                         175
La Barceloneta                      160
Sant Andreu                         133
Camp d'en Grassot i Gràcia Nova     122
Sant Gervasi - Galvany              121


In [13]:
cat_fea = ['neighbourhood','room_type', 'property_type', 'cancellation_policy', 'property_type vs room_type']
final_data = pd.get_dummies(df, columns=cat_fea)
final_data.head(2)

,name,host_id,host_name,latitude,longitude,price,number_of_reviews,availability_365,accommodates,review_scores_rating,...,property_type_Serviced apartment,property_type_Tiny house,property_type_Townhouse,property_type_Villa,cancellation_policy_flexible,cancellation_policy_moderate,cancellation_policy_strict,property_type vs room_type_Apartment with All Rooms/Full Property,property_type vs room_type_Apartment with Private room,property_type vs room_type_Others
0,Flat with Sunny Terrace,71615,Mireia And Maria,41.40889,2.18555,130.0,1,182,6,80.0,...,0,0,0,0,1,0,0,1,0,0
1,Huge flat for 8 people close to Sagrada Familia,71615,Mireia And Maria,41.40420,2.17306,60.0,15,129,8,87.0,...,0,0,0,0,0,0,1,1,0,0


In [14]:
for i in cat_fea:
  print(i, df[i].unique().shape)

neighbourhood (31,)
room_type (3,)
property_type (26,)
cancellation_policy (3,)
property_type vs room_type (3,)


In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X, y = final_data.drop(columns=['price','name','host_id','host_name']), np.log(final_data['price'])

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Modeling

In [19]:
from sklearn.ensemble import RandomForestRegressor

In [25]:
xgb_instance = XGBRegressor(n_estimators=1024,
                            learning_rate=0.1, 
                            subsample=0.8, 
                            colsample_bytree=1, 
                            scale_pos_weight=10)
                    
model = xgb_instance.fit(X_train, y_train)

In [26]:
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)

(0.24301674615729565, 0.6397517823873954)

In [37]:
x = pd.DataFrame()
x['features'] = X_train.columns
x['score'] = model.feature_importances_
x.sort_values(by='score', ascending=False).head(10)

,features,score
37,room_type_All Rooms/Full Property,0.415408
57,property_type_Hotel,0.146143
60,property_type_Nature lodge,0.055745
45,property_type_Boutique hotel,0.042736
4,accommodates,0.029892
39,room_type_Shared room,0.021042
56,property_type_Hostel,0.017147
44,property_type_Boat,0.016093
62,property_type_Serviced apartment,0.014674
43,property_type_Bed and breakfast,0.011989


In [21]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
from tqdm.notebook import tqdm

In [ ]:
mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)

(0.3531949143461075, 0.4764235783131362)

In [ ]:
table = pd.DataFrame(columns = ["#Trees",  "LR", "Subsample", "% Features", "Weight of Default"])

row = 0
for numtrees in tqdm([256, 512, 768, 1024]):
    for LR in tqdm([0.01, 0.1]):  
        for Subsample in [0.5, 0.8]:
            for colsample_bytree in [0.5, 1]:
                for scale_pos_weight in [1, 5, 10]:
                    xgb_instance = XGBRegressor(n_estimators=numtrees, 
                                                learning_rate=LR, 
                                                subsample=Subsample, 
                                                colsample_bytree=colsample_bytree, 
                                                scale_pos_weight=scale_pos_weight)
                    
                    model = xgb_instance.fit(X_train, y_train)

                    table.loc[row, "#Trees"] = numtrees
                    table.loc[row, "LR"] = LR
                    table.loc[row, "Subsample"] = Subsample
                    table.loc[row, "% Features"] = colsample_bytree
                    table.loc[row, "Weight of Default"] = scale_pos_weight
                    table.loc[row, "R2_train"] = r2_score(y_train, model.predict(X_train))
                    table.loc[row, "R2_test"] = r2_score(y_test, model.predict(X_test))
                    row = row + 1

table

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

,#Trees,LR,Subsample,% Features,Weight of Default,R2_train,R2_test
0,256,0.01,0.5,0.5,1,0.490566,0.466303
1,256,0.01,0.5,0.5,5,0.490566,0.466303
2,256,0.01,0.5,0.5,10,0.490566,0.466303
3,256,0.01,0.5,1,1,0.512626,0.487182
4,256,0.01,0.5,1,5,0.512626,0.487182
...,...,...,...,...,...,...,...
91,1024,0.1,0.8,0.5,5,0.963163,0.636511
92,1024,0.1,0.8,0.5,10,0.963163,0.636511
93,1024,0.1,0.8,1,1,0.976259,0.639752
94,1024,0.1,0.8,1,5,0.976259,0.639752


In [ ]:
table = pd.DataFrame(columns = ["#Trees",  "LR", "Subsample", "% Features", "Weight of Default"])

row = 0
for numtrees in tqdm([256, 512, 768, 1024]):
    for mss in tqdm([2,4,8,16,32]):
        for msl in [1,2,4,8]:
            xgb_instance = XGBRegressor(n_estimators=numtrees,
                                        criterion='squared_error',
                                        max_depth=None,
                                        min_samples_split=mss,
                                        min_samples_leaf=msl,
                                        min_weight_fraction_leaf=0.0)
            
            model = xgb_instance.fit(X_train, y_train)

            table.loc[row, "#Trees"] = numtrees
            table.loc[row, "LR"] = LR
            table.loc[row, "Subsample"] = Subsample
            table.loc[row, "% Features"] = colsample_bytree
            table.loc[row, "Weight of Default"] = scale_pos_weight
            table.loc[row, "R2_train"] = r2_score(y_train, model.predict(X_train))
            table.loc[row, "R2_test"] = r2_score(y_test, model.predict(X_test))
            row = row + 1

table

In [27]:
x = pd.DataFrame()
x['Model'] = ['Linear Regressor', 'Random Forest', 'XGBoost']
x['MSE'] = [1.2546, 0.3529, 0.2431]
x['R-Squared'] = [0.2794, 0.5164, 0.7376]
x.head()

,Model,MSE,R-Squared
0,Linear Regressor,1.2546,0.2794
1,Random Forest,0.3529,0.5164
2,XGBoost,0.2431,0.7376


In [28]:
import plotly.graph_objects as go

models = ['Linear Regressor', 'Random Forest', 'XGBoost']
mse = [1.2546, 0.3529, 0.2431]
r_squared = [0.2794, 0.5164, 0.7376]

fig = go.Figure()

fig.add_trace(go.Bar(
    y=models,
    x=mse,
    name='MSE',
    orientation='h',
    marker=dict(
        color='rgba(255, 0, 0, 0.6)',
        line=dict(color='rgba(255, 0, 0, 1.0)', width=2)
    )
))

fig.add_trace(go.Bar(
    y=models,
    x=r_squared,
    name='R-Squared',
    orientation='h',
    marker=dict(
        color='rgba(0, 0, 0, 0.6)',
        line=dict(color='rgba(0, 0, 0, 1.0)', width=2)
    )
))

fig.update_layout(
    title='Model Comparison',
    xaxis_title='Metric Value',
    yaxis_title='Model',
    barmode='group',
    bargap=0.1,
    bargroupgap=0.2,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    font=dict(color='white'),
)

fig.show(renderer="colab")